In [1]:
import pandas as pd
import sys # 별도 파일로 보관중인 패키지 불러오기용(환경변수 추가)
sys.path.append('C:\\python_source')

import NERP_PI_LC

sessions = NERP_PI_LC.check_and_open_sap('System name', 'ID', 'PW',3)
session = sessions[1]

def crawl_hscode_with_sr(session, sr_no):
    dict_result = dict()
    session.StartTransaction('ZLED50110_D')
    session.findById("wnd[0]/usr/ctxtG_TKNUM").text = sr_no
    session.findById("wnd[0]").sendVKey(0)

    # Sales, Plant
    dict_result['SalesOrg'] = session.findById("wnd[0]/usr/txtGT_SR-ZZVKORG").text
    dict_result['PlantCode'] = session.findById("wnd[0]/usr/txtGT_SR-ZZWERKS").text
    dict_result['PlaneName'] = session.findById("wnd[0]/usr/txtGT_SR-WERKS_NAME").text

    # Port
    session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC1").select()
    dict_result['POL'] = session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC1/ssubSUB_SCA:SAPMZLED50110:0310/txtG_LOAD_TEXT").text
    dict_result['F.Dest'] = session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC1/ssubSUB_SCA:SAPMZLED50110:0310/txtG_FDEST_TEXT").text

    # HSCODE
    session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC4").select()
    temp_set_hscode = set()
    temp_set_itemname = set()
    total_row = session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340A").RowCount
    for idx in range((total_row)):
        session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340A").verticalScrollbar.position = idx
        temp_set_itemname.add(session.findById(f"wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340A/txtGT_MAT-GOODS_DESC1[13,{0}]").text)
        temp_set_hscode.add(session.findById(f"wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340A/txtGT_MAT-ZZSTAWN[14,{0}]").text)
    
    # 여러건의 데이터를 한 줄의 데이터로 합치기
    dict_result['HSCODE'] = ', '.join(temp_set_hscode)
    dict_result['Description'] = ', '.join(temp_set_itemname)

    session.findById("wnd[0]").sendVKey(3)

    return dict_result

## 1건 확인용

In [2]:
# SR 1건 확인
sr_no = 'INV'
data_all = crawl_hscode_with_sr(session, sr_no)

# 엑셀저장용으로 데이터프레임 생성
df = pd.DataFrame(columns=[list(data_all.keys())])
for each_col in data_all.keys():
    df.loc[sr_no, each_col] = data_all[each_col]

# 결과출력
print('*', sr_no)
for key in data_all.keys():
    print(key+' :', data_all[key])

* INV
SalesOrg : 9999
PlantCode : P000
PlaneName : Dummy Plant
POL : Gwangyang,South Korea
F.Dest : Tunis,Tunisia
HSCODE : 8415102010, 8415102020, 8415900000
Description : REFRIDGERATER


## 여러건 확인용
* sr_no의 '''와 ''' 사이에 붙여넣고 실행 (엑셀 등에서 표를 바로 붙여넣기 편하도록 설계)

In [3]:
# SR 여러건 확인
sr_no = '''
INV1
INV2
'''.split('\n')

sr_no = list(set(sr_no))
sr_no.remove('')

#빈 DataFrame 생성
df = pd.DataFrame(columns=[list(crawl_hscode_with_sr(session, sr_no[0]).keys())])

for idx, each_sr in enumerate(sr_no):
    data_all = crawl_hscode_with_sr(session, each_sr)
    #set_hscode = crawl_hscode_with_sr(session, each_sr)
    #df = pd.concat([df, pd.DataFrame({'SR NO': each_sr, 'HSCODE': data_all})], ignore_index=True)
    for each_col in data_all.keys():
        df.loc[each_sr,each_col] = data_all[each_col]

df


,SalesOrg,PlantCode,PlaneName,POL,F.Dest,HSCODE,Description
INV1,R002,P000,DUMMY PLANT1,"Busan,South Korea","Matadi,Dem. Rep. Congo","8415900000, 8415102010",REFRIDGERATOR
INV2,Q001,P000,DUMMY PLANT2,"Incheon Intl Apt/Seoul,South Korea","Dhaka,Bangladesh",8542391000,WASHING MACHINE


## 확인한 내용 엑셀로 저장

In [4]:
# 엑셀파일로 저장
from datetime import datetime

df.to_excel(f'shipment_info_summary_{datetime.strftime(datetime.today(),"%Y%m%d")}.xlsx')